In [1]:
import random
from collections import Counter

DECK_COMPLETE = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']

#hit on 8 or less
#stay on 17 or more
#line determined by player's hand score, starting on a 9
#row determined by dealer's hand score, starting on a 2
#H for hit, D for double, S for stand
BASIC_STRATEGY = [['H','D','D','D','D','H','H','H','H','H'],
                  ['D','D','D','D','D','D','D','D','H','H'],
                  ['D','D','D','D','D','D','D','D','D','H'],
                  ['H','H','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H']]

In [2]:
def rand_card(deck_func):
    card = random.choice(deck_func)
    deck_func.remove(card)
    return card


def score(cards):
    score = 0
    possible_hands = []
    aces = 0

    for x in cards:
        if type(x) == int:
            score += x
        elif x in ["J", "Q", "K"]:
            score += 10
        else:
            score += 1
            aces += 1

    for i in range(aces + 1):
        if (score + i * 10) <= 21:
            possible_hands.append(score + i * 10)

    if (possible_hands) == []:
        return -1
    return max(possible_hands)

In [3]:
def bj_round(hit_on_17=False):
    
    deck_current = DECK_COMPLETE.copy()
    
    player_cards = [rand_card(deck_func=deck_current), rand_card(deck_func=deck_current)]
    player_score = score(player_cards)
    dealer_cards = [rand_card(deck_func=deck_current)]
    dealer_score = score(dealer_cards)
    
    if(player_score==21):
        return ('Win', 21)
    
    game_state = 'Ongoing'
    strategy = ''
    double_flag = 0
    
    while game_state == 'Ongoing':
        if(player_score<=8):
            strategy = 'H'
        elif(player_score>=17+hit_on_17):
            strategy = 'S'
        else:
            strategy = BASIC_STRATEGY[(player_score-9)][(dealer_score-2)]

        if(strategy=='D'):
            double_flag = 1
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if(player_score==-1):
                game_state='Loss'
            else:
                game_state='S'
        if(strategy=='H'):
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if(player_score==-1):
                game_state='Loss'
        if(strategy=='S'):
            game_state='S'
        
    if game_state == 'Loss':
        return ('Loss', player_score)
    
    while((dealer_score<17+hit_on_17) ^ (dealer_score ==-1)):
        dealer_cards.append(rand_card(deck_func=deck_current))
        dealer_score = score(dealer_cards)
    
    if((player_score > dealer_score) or (dealer_score>21)):
        return ('Win', player_score)
    elif(player_score == dealer_score):
        return ('Draw', player_score)
    else:
        return ('Loss', player_score)

In [4]:
results = {"Win": [], "Draw": [], "Loss": []}

for _ in range(1000000):
    wdl, total = bj_round()
    results[wdl].append(total)

In [5]:
for key, values in results.items():
    print(f"{key}:")

    total = dict(Counter(values))

    sorted_dict = dict(sorted(total.items(), reverse=True))

    for key, value in sorted_dict.items():
        print(f"{key}:{round(value*100/sum(sorted_dict.values()), 3)}%")
    print("\n")

Win:
21:21.923%
20:25.154%
19:14.764%
18:11.438%
17:7.887%
16:3.718%
15:3.919%
14:4.024%
13:4.267%
12:2.82%
11:0.087%


Draw:
21:7.987%
20:32.815%
19:18.27%
18:19.478%
17:21.449%


Loss:
20:4.017%
19:7.494%
18:11.045%
17:15.51%
16:5.4%
15:5.591%
14:5.697%
13:5.988%
12:3.823%
11:0.108%
-1:35.327%




In [6]:
def bj_round_till_win(cum_loss=0):
    deck_current = DECK_COMPLETE.copy()

    player_cards = [
        rand_card(deck_func=deck_current),
        rand_card(deck_func=deck_current),
    ]
    player_score = score(player_cards)
    dealer_cards = [rand_card(deck_func=deck_current)]
    dealer_score = score(dealer_cards)

    if player_score == 21:
        return ("Win", 21, 21 + cum_loss)

    game_state = "Ongoing"
    strategy = ""

    while game_state == "Ongoing":
        if player_score <= 8:
            strategy = "H"
        elif player_score >= 17:
            strategy = "S"
        else:
            strategy = BASIC_STRATEGY[(player_score - 9)][(dealer_score - 2)]

        if strategy == "D":
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if player_score == -1:
                cum_loss += 1
                game_state = "L"
            else:
                cum_loss += -1
                game_state = "S"
        if strategy == "H":
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if player_score == -1:
                game_state = "L"
        if strategy == "S":
            game_state = "S"

    if game_state == "Loss":
        return bj_round_till_win(cum_loss=cum_loss - 2)

    while (dealer_score < 17) ^ (dealer_score == -1):
        dealer_cards.append(rand_card(deck_func=deck_current))
        dealer_score = score(dealer_cards)

    if (player_score > dealer_score) or (dealer_score > 21):
        return ("Win", player_score, player_score + cum_loss)
    elif player_score == dealer_score:
        return bj_round_till_win(cum_loss=cum_loss)
    else:
        return bj_round_till_win(cum_loss=cum_loss - 2)


bj_round_till_win()

('Win', 19, 19)

In [16]:
#min score 7
for _ in range(4):
    player = []
    for _ in range(6):
        player.append(max(bj_round_till_win()[2] - 3, 7))
    print(sorted(player, reverse=True))

[17, 17, 17, 15, 15, 13]
[18, 16, 15, 15, 15, 14]
[16, 14, 14, 14, 14, 13]
[15, 15, 15, 15, 14, 13]


In [8]:
def bj_round_till_win_2(cum_loss=0):
    deck_current = DECK_COMPLETE.copy()

    player_cards = [
        rand_card(deck_func=deck_current),
        rand_card(deck_func=deck_current),
    ]
    player_score = score(player_cards)
    dealer_cards = [rand_card(deck_func=deck_current)]
    dealer_score = score(dealer_cards)

    if player_score == 21:
        return ("Win", 21, 21 - cum_loss)

    game_state = "Ongoing"
    strategy = ""
    double_flag = 0

    while game_state == "Ongoing":
        if player_score <= 8:
            strategy = "H"
        elif player_score >= 17:
            strategy = "S"
        else:
            strategy = BASIC_STRATEGY[(player_score - 9)][(dealer_score - 2)]

        if strategy == "D":
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if player_score == -1:
                cum_loss += 1
                game_state = "L"
            else:
                cum_loss += -1
                game_state = "S"
        if strategy == "H":
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if player_score == -1:
                game_state = "L"
        if strategy == "S":
            game_state = "S"

    if game_state == "Loss":
        return ("L", cum_loss - 1)

    while (dealer_score < 17) ^ (dealer_score == -1):
        dealer_cards.append(rand_card(deck_func=deck_current))
        dealer_score = score(dealer_cards)

    if (player_score > dealer_score) or (dealer_score > 21):
        return ("Win", cum_loss, player_score - cum_loss)
    elif player_score == dealer_score:
        return ("D", cum_loss)
    else:
        return ("L", cum_loss + 1)


bj_round_till_win_2()


('L', 1)

In [21]:
#min score 7
for _ in range(4):
    cum_loss = 0
    player = []
    while len(player) < 6:

        game = bj_round_till_win_2(cum_loss)
        cum_loss = game[1]

        if game[0] == "Win":
            player.append(max(game[2] - 3, 7))

    print(sorted(player, reverse=True))

[16, 14, 11, 9, 9, 8]
[17, 17, 16, 12, 7, 7]
[17, 7, 7, 7, 7, 7]
[18, 16, 16, 14, 14, 14]
